In [1]:
import re
import pandas as pd

In [2]:
raw_7_df = pd.read_excel(
    "./../raw_data/제_7_전국동시지방선거_개표/20180619-7지선-01-(시도지사)_읍면동별개표자료.xlsx",
    header=[0],
)

In [3]:
df = raw_7_df.copy()

In [4]:
# get splited_index
_df = df.dropna(subset=["선거종류"])
index_zip_list = list(zip(_df.index, _df.index[1:]))

In [5]:
_replace_nan = lambda x: "" if type(x) != str else x
_replace_unnamed = lambda x: "후보자별 득표수" if x.startswith("Unnamed") else x
_replace_escape = lambda x: x.replace("_x000D_\n", "_")
_replace = lambda x: _replace_escape(_replace_unnamed(_replace_nan(x)))

_join_strings = lambda x: "_".join(x).strip("_")

func = lambda x: _join_strings(map(_replace, x))

In [6]:
def replace_position(df, lib, con, etc):
    """
    lib : 진보당
    con : 보수당
    etc : 나머지당
    """
    df["lib"] = df.loc[:, lib].sum(axis=1)
    df["con"] = df.loc[:, con].sum(axis=1)
    df["etc"] = df.loc[:, etc].sum(axis=1)
    df.drop(columns=(lib + con + etc), inplace=True)
    return df

In [7]:
partial_df_list = list()
for index_zip in index_zip_list:
    # partial data
    partial_df = df.iloc[index_zip[0] : index_zip[1], :]

    # preproc columns
    multi_cols = [(col_a, col_b) for col_a, col_b in zip(partial_df.columns, partial_df.iloc[0, :])]

    flatten_cols = list(map(func, multi_cols))
    partial_df.columns = flatten_cols

    # drop duplicated data
    partial_df = partial_df.iloc[1:, 2:]

    # filter agg / etcs
    partial_df = partial_df[
        (partial_df["시도명"] != "합계")
        & (partial_df["구시군명"] != "합계")
        & ~(
            partial_df["읍면동명"].isin(
                ["합계", "거소투표", "관외사전투표", "재외투표", "잘못 투입·구분된 투표지"]
            )
        )
        & (partial_df["구분"] == "소계")
    ]
    partial_df = partial_df.loc[:, ~partial_df.columns.duplicated()]

    # renaming
    rename_dict = {
        "시도명": "loc_1",
        "구시군명": "loc_2",
        "읍면동명": "loc_3",
    }

    partial_df.rename(columns=rename_dict, inplace=True)
    partial_df.drop(columns=["구분", "후보자별 득표수"], inplace=True, errors="ignore")

    #
    votes = list(filter(lambda x: x.startswith("후보자별 득표수"), partial_df.columns))
    partial_df.loc[:, votes] = partial_df.loc[:, votes].astype(int)
    lib = list(filter(lambda x: x.startswith("후보자별 득표수_더불어민주당"), partial_df.columns))
    con = list(filter(lambda x: x.startswith("후보자별 득표수_자유한국당"), partial_df.columns))
    etc = list(set(votes) - (set(lib) | set(con)))
    partial_df = replace_position(partial_df, lib, con, etc)

    # save
    argument = partial_df["loc_1"].iloc[0]
    partial_df_list.append(partial_df)

In [8]:
시장도지사_7 = pd.concat(partial_df_list, axis=0)
시장도지사_7.head()

,loc_1,loc_2,loc_3,선거인수,투표수,무효투표수,기권수,lib,con,etc
5,서울특별시,종로구,청운효자동,9719.0,6057.0,77.0,3662.0,3025,1420,7515
8,서울특별시,종로구,사직동,8106.0,4972.0,46.0,3134.0,2097,1453,6302
11,서울특별시,종로구,삼청동,2307.0,1284.0,10.0,1023.0,583,376,1589
14,서울특별시,종로구,부암동,7997.0,4780.0,67.0,3217.0,2277,1255,5894
17,서울특별시,종로구,평창동,15012.0,8820.0,78.0,6192.0,3702,2814,10968


In [9]:
시장도지사_7.to_csv("./1st_preproc_data/시장도지사_7.csv")

In [10]:
raw_8_df = pd.read_excel(
    "./../raw_data/제_8_전국동시지방선거_개표/시도지사선거.xlsx", header=[0, 1], thousands=","
)

In [11]:
df = raw_8_df.copy()

In [12]:
# preproc columns
_replace_unnamed = lambda x: "" if x.startswith("Unnamed") else x
_replace_candidate = lambda x: "" if re.match("^후보\d+", x) else x
_join_strings = lambda x: "_".join(x).strip("_")

func = lambda x: _join_strings(map(_replace_unnamed, map(_replace_candidate, x)))

flatten_cols = list(map(func, df.columns))
df.columns = flatten_cols

In [13]:
# get splited_index
index_zip_list = list()
for arg in df["선거구명"].unique():
    _index = df[df["선거구명"] == arg].index
    index_zip_list.append((min(_index), max(_index)))

In [14]:
_replace_nan = lambda x: "" if type(x) != str else x
_replace_unnamed = lambda x: "후보자별 득표수" if x.startswith("Unnamed") else x
_replace_escape = lambda x: x.replace("\n", "_")
_replace = lambda x: _replace_escape(_replace_unnamed(_replace_nan(x)))

_join_strings = lambda x: "_".join(x).strip("_")

func = lambda x: _join_strings(map(_replace, x))

In [15]:
partial_df_list = list()
for index_zip in index_zip_list:
    # partial data
    partial_df = df.iloc[index_zip[0] : index_zip[1] + 1, :]

    # preproc columns
    multi_cols = [(col_a, col_b) for col_a, col_b in zip(partial_df.columns, partial_df.iloc[0, :])]
    flatten_cols = list(map(func, multi_cols))
    partial_df.columns = flatten_cols

    # drop duplicated data
    partial_df = partial_df.iloc[1:, :]

    # filter agg / etcs
    partial_df = partial_df[
        ~(
            partial_df["읍면동명"].isin(
                ["합계", "거소투표", "관외사전투표", "잘못 투입·구분된 투표지"]
            )
        )
        & (partial_df["구분"] == "소계")
    ]
    partial_df = partial_df.loc[:, ~partial_df.columns.duplicated()]

    #
    partial_df.drop(columns=["구분", "후보자별 득표수"], inplace=True, errors="ignore")
    partial_df.columns = ["loc_1", "loc_2", "loc_3"] + list(partial_df.columns)[3:]
    partial_df.columns = [col.replace("_선거인수", "") for col in partial_df.columns]

    #
    votes = list(filter(lambda x: x.startswith("후보자별 득표수"), partial_df.columns))
    partial_df.loc[:, votes] = partial_df.loc[:, votes].astype(int)
    lib = list(filter(lambda x: x.startswith("후보자별 득표수_더불어민주당"), partial_df.columns))
    con = list(filter(lambda x: x.startswith("후보자별 득표수_국민의힘"), partial_df.columns))
    etc = list(set(votes) - (set(lib) | set(con)))
    partial_df = replace_position(partial_df, lib, con, etc)
    # save
    argument = partial_df["loc_1"].iloc[0]
    partial_df_list.append(partial_df)

In [16]:
시장도지사_8 = pd.concat(partial_df_list, axis=0)
시장도지사_8.head()

,loc_1,loc_2,loc_3,선거인수,투표수,계,무효투표수,기권수,lib,con,etc
4,서울특별시,종로구,청운효자동,9179.0,4887.0,4849.0,38.0,4292.0,2094,2630,125
7,서울특별시,종로구,사직동,8057.0,4452.0,4426.0,26.0,3605.0,1523,2785,118
10,서울특별시,종로구,삼청동,1962.0,1004.0,1001.0,3.0,958.0,388,591,22
13,서울특별시,종로구,부암동,7693.0,4137.0,4100.0,37.0,3556.0,1665,2346,89
16,서울특별시,종로구,평창동,14730.0,7931.0,7881.0,50.0,6799.0,2594,5143,144


In [17]:
시장도지사_8.to_csv("./1st_preproc_data/시장도지사_8.csv")